In [1]:
using Distributions
mean = [2.,3.]
C = [0.2 0; 0 0.3]
d = MvNormal(mean, C)
x = rand(d, 2000)

2×2000 Matrix{Float64}:
 1.5258   2.22738  1.53614  2.28665  …  2.15072  1.96453  2.4128   1.72141
 2.96187  2.90154  3.27173  3.2597      2.99756  2.68633  3.31022  2.89368